In [6]:
import tensorflow as tf
import numpy as np


print("=== TensorFlow & Keras 설치 확인 ===")

# 1. 버전 확인
print(f"TensorFlow 버전: {tf.__version__}")
print(f"Keras 버전: {tf.keras.__version__}")

# 2. GPU 사용 가능 여부 (Mac Silicon Metal 지원)
gpus = tf.config.list_physical_devices('GPU')
print(f"GPU 사용 가능: {gpus}")
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")

# Mac Silicon Metal GPU 확인
metal_available = False
try:
    import tensorflow_metal
    metal_available = True
    print("✅ tensorflow-metal 설치됨 (Mac Silicon GPU 가속 가능)")
except ImportError:
    # GPU가 있다면 tensorflow-metal이 백그라운드에서 작동 중일 수 있음
    if gpus:
        print("⚠️ tensorflow-metal import 실패했지만 GPU 감지됨 (정상 작동 가능)")
        metal_available = True
    else:
        print("❌ tensorflow-metal 미설치 (uv sync로 설치 필요)")

# Metal GPU 메모리 설정 (Mac Silicon 전용)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU 메모리 동적 할당 설정 완료")
    except:
        print("⚠️ GPU 메모리 설정 실패 (이미 초기화됨)")

# 3. 간단한 연산 테스트
print("\n=== 기본 연산 테스트 ===")
a = tf.constant([1, 2, 3])
b = tf.constant([4, 5, 6])
result = tf.add(a, b)
print(f"[1,2,3] + [4,5,6] = {result.numpy()}")

# 4. 간단한 모델 생성 및 컴파일 테스트
print("\n=== 모델 생성 테스트 ===")
try:
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu', input_shape=(5,)),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    print("✅ 모델 생성 및 컴파일 성공")
    
    # 더미 데이터로 예측 테스트
    dummy_input = np.random.random((1, 5))
    prediction = model.predict(dummy_input, verbose=0)
    print(f"✅ 예측 테스트 성공: {prediction[0][0]:.4f}")
    
except Exception as e:
    print(f"❌ 모델 테스트 실패: {e}")

# 추가 검증
if tf.__version__:
    print(f"✅ TensorFlow 버전: {tf.__version__}")
    print(f"✅ Keras 버전: {tf.keras.__version__}")
    print(f"✅ GPU 사용 가능: {len(tf.config.list_physical_devices('GPU')) > 0}")
else:
    print("❌ TensorFlow가 제대로 설치되지 않았습니다.")


=== TensorFlow & Keras 설치 확인 ===
TensorFlow 버전: 2.17.1
Keras 버전: 3.5.0
GPU 사용 가능: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Built with CUDA: False
⚠️ tensorflow-metal import 실패했지만 GPU 감지됨 (정상 작동 가능)
✅ GPU 메모리 동적 할당 설정 완료

=== 기본 연산 테스트 ===
[1,2,3] + [4,5,6] = [5 7 9]

=== 모델 생성 테스트 ===
✅ 모델 생성 및 컴파일 성공
✅ 예측 테스트 성공: -0.3863
✅ TensorFlow 버전: 2.17.1
✅ Keras 버전: 3.5.0
✅ GPU 사용 가능: True


In [4]:
# Metal GPU 가속 성능 테스트
import tensorflow as tf
import time

print("=== Mac Silicon Metal GPU 성능 테스트 ===")

# GPU 장치 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU 장치: {gpus[0].name}")
    
    # 큰 행렬 연산으로 GPU 가속 테스트
    print("\n🔥 GPU 가속 테스트 중...")
    
    with tf.device('/GPU:0'):
        # 큰 행렬 생성
        a = tf.random.normal([2000, 2000])
        b = tf.random.normal([2000, 2000])
        
        # GPU에서 행렬 곱셈 수행
        start_time = time.time()
        result = tf.matmul(a, b)
        gpu_time = time.time() - start_time
        
        print(f"✅ GPU 연산 완료: {gpu_time:.4f}초")
        print(f"✅ 결과 shape: {result.shape}")
        print(f"✅ Metal GPU 가속 정상 작동!")
        
    # CPU와 비교 (작은 행렬로)
    with tf.device('/CPU:0'):
        a_cpu = tf.random.normal([1000, 1000])
        b_cpu = tf.random.normal([1000, 1000])
        
        start_time = time.time()
        result_cpu = tf.matmul(a_cpu, b_cpu)
        cpu_time = time.time() - start_time
        
        print(f"📊 CPU 연산 시간 (1000x1000): {cpu_time:.4f}초")
        
else:
    print("❌ GPU를 찾을 수 없습니다.")


=== Mac Silicon Metal GPU 성능 테스트 ===
✅ GPU 장치: /physical_device:GPU:0

🔥 GPU 가속 테스트 중...
✅ GPU 연산 완료: 0.0404초
✅ 결과 shape: (2000, 2000)
✅ Metal GPU 가속 정상 작동!
📊 CPU 연산 시간 (1000x1000): 0.0025초
